# Routing profiles

PyVRP supports routing problems with multiple vehicle types.
These vehicle types can differ in many ways, some of which we have already seen in previous tutorials.
Another way in which vehicle types can differ is in the distances and durations of the paths they need to travel between locations.

In this notebook we explore PyVRP's *profiles*, which can be used to model these differences.
The profile feature is particularly helpful for problems with mixed fleets of trucks, cars, and/or bicycles, as well as access restrictions.

## Mixed fleets

TODO

## Zone restrictions

Let us continue with a variant showing how to model zone restrictions, where some vehicles are not allowed to visit clients located inside a particular area.
Such restrictions commonly apply in urban environments with emission zones, where several types of (heavy) trucks may not enter.
We will add one regular vehicle type to the model that can enter the restricted zone.
Additionally, we will consider a vehicle type that cannot enter the restricted zone, and has to travel from the first to the second depot.

Suppose we have a rectangular zone defined by the following `(x, y)` coordinates.

In [ ]:
ZONE = ((500, 125), (850, 275))


def in_zone(client) -> bool:
    return (
        ZONE[0][0] <= client.x <= ZONE[1][0]
        and ZONE[0][1] <= client.y <= ZONE[1][1]
    )

We can now set up a `Model` as follows, using routing profiles to restrict which vehicle types can enter the zone to visit clients there.

In [ ]:
m = Model()

depot1 = m.add_depot(x=COORDS[0][0], y=COORDS[0][1])
depot2 = m.add_depot(x=COORDS[1][0], y=COORDS[1][1])

regular = m.add_profile(name="regular")
m.add_vehicle_type(
    2,
    start_depot=depot1,
    end_depot=depot1,
    tw_early=TIME_WINDOWS[0][0],
    tw_late=TIME_WINDOWS[0][1],
    profile=regular,
)

restricted = m.add_profile(name="restricted")
m.add_vehicle_type(
    2,
    start_depot=depot1,
    end_depot=depot2,
    tw_early=TIME_WINDOWS[1][0],
    tw_late=TIME_WINDOWS[1][1],
    profile=restricted,
)

for idx in range(2, len(COORDS)):
    m.add_client(
        x=COORDS[idx][0],
        y=COORDS[idx][1],
        tw_early=TIME_WINDOWS[idx][0],
        tw_late=TIME_WINDOWS[idx][1],
    )

for frm_idx, frm in enumerate(m.locations):
    for to_idx, to in enumerate(m.locations):
        distance = abs(frm.x - to.x) + abs(frm.y - to.y)  # Manhattan
        duration = DURATION_MATRIX[frm_idx][to_idx]

        # Edges without a specific profile assignment are added to all
        # profiles, unless a profile-specific edge overrides them.
        m.add_edge(frm, to, distance=distance, duration=duration)

        if frm_idx != to_idx and in_zone(to):
            # Here we specify an edge with a high distance and duration
            # for the restricted profile. This ensures vehicles with
            # that profile do not travel over this edge.
            m.add_edge(
                frm,
                to,
                distance=1_000,
                duration=1_000,
                profile=restricted,
            )

In [ ]:
res = m.solve(stop=MaxRuntime(1), display=False)  # one second
print(res)

In [ ]:
_, ax = plt.subplots(figsize=(8, 8))
plot_solution(res.best, m.data(), ax=ax)

# Highlight the restricted zone.
ax.fill_between(
    [ZONE[0][0], ZONE[1][0]],
    ZONE[0][1],
    ZONE[1][1],
    color="red",
    alpha=0.15,
);

## Conclusion

TODO